# AR6 calibration of FaIR 2.1

Calibration
- use the exact aerosol indirect forcing relationship from Smith et al. (2021). 
- aerosol direct is from AR6.
- three layer model for climate response.
- include overlap of the major GHGs.
- prognostic equation for land use related forcing (e.g. from FaIR 1.6).
- ozone relationship from FaIR 1.6 used in AR6.
- interactive methane lifetime (NEW!)

TODO:
- solar trend
- vary pre-industrial concentrations of CO2

CONSTRAINT:
- Markov-chain Monte Carlo based on AR6 assessed ranges.

## Basic imports

In [ ]:
import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import time
import scipy.stats
from tqdm import tqdm

from fair21 import (
    SpeciesID, Category, Config, Species, RunMode, Scenario, ClimateResponse, RunConfig, CH4LifetimeMethod, FAIR
)
from fair21.defaults import species_config_from_default
from fair21.energy_balance_model import EnergyBalanceModel

## Set up problem

In [ ]:
species_ids = {
    # Greenhouse gases and precursors
    'CO2_FFI': SpeciesID('CO2 fossil fuel and industrial', Category.CO2_FFI),
    'CO2_AFOLU': SpeciesID('CO2 AFOLU', Category.CO2_AFOLU),
    'CO2': SpeciesID('CO2', Category.CO2),
    'CH4': SpeciesID('CH4', Category.CH4),
    'N2O': SpeciesID('N2O', Category.N2O),
    'CFC-11': SpeciesID('CFC-11', Category.CFC_11),
    'CFC-12': SpeciesID('CFC-12', Category.OTHER_HALOGEN),
    'CFC-113': SpeciesID('CFC-113', Category.OTHER_HALOGEN),
    'CFC-114': SpeciesID('CFC-114', Category.OTHER_HALOGEN),
    'CFC-115': SpeciesID('CFC-115', Category.OTHER_HALOGEN),
    'HCFC-22': SpeciesID('HCFC-22', Category.OTHER_HALOGEN),
    'HCFC-141b': SpeciesID('HCFC-141b', Category.OTHER_HALOGEN),
    'HCFC-142b': SpeciesID('HCFC-142b', Category.OTHER_HALOGEN),
    'CCl4': SpeciesID('CCl4', Category.OTHER_HALOGEN),
    'CHCl3': SpeciesID('CHCl3', Category.OTHER_HALOGEN),
    'CH2Cl2': SpeciesID('CH2Cl2', Category.OTHER_HALOGEN),
    'CH3Cl': SpeciesID('CH3Cl', Category.OTHER_HALOGEN),
    'CH3CCl3': SpeciesID('CH3CCl3', Category.OTHER_HALOGEN),
    'CH3Br': SpeciesID('CH3Br', Category.OTHER_HALOGEN),
    'Halon-1211': SpeciesID('Halon-1211', Category.OTHER_HALOGEN),
    'Halon-1301': SpeciesID('Halon-1301', Category.OTHER_HALOGEN),
    'Halon-2402': SpeciesID('Halon-2402', Category.OTHER_HALOGEN),
    'CF4': SpeciesID('CF4', Category.F_GAS),
    'C2F6': SpeciesID('C2F6', Category.F_GAS),
    'C3F8': SpeciesID('C3F8', Category.F_GAS),
    'c-C4F8': SpeciesID('C-C4F8', Category.F_GAS),
    'C4F10': SpeciesID('C4F10', Category.F_GAS),
    'C5F12': SpeciesID('C5F12', Category.F_GAS),
    'C6F14': SpeciesID('C6F14', Category.F_GAS),
    'C7F16': SpeciesID('C7F16', Category.F_GAS),
    'C8F18': SpeciesID('C8F18', Category.F_GAS),
    'HFC-125': SpeciesID('HFC-125', Category.F_GAS),
    'HFC-134a': SpeciesID('HFC-134a', Category.F_GAS),
    'HFC-143a': SpeciesID('HFC-143a', Category.F_GAS),
    'HFC-152a': SpeciesID('HFC-152a', Category.F_GAS),
    'HFC-227ea': SpeciesID('HFC-227ea', Category.F_GAS),
    'HFC-23': SpeciesID('HFC-23', Category.F_GAS),
    'HFC-236fa': SpeciesID('HFC-236fa', Category.F_GAS),
    'HFC-245fa': SpeciesID('HFC-245fa', Category.F_GAS),
    'HFC-32': SpeciesID('HFC-32', Category.F_GAS),
    'HFC-365mfc': SpeciesID('HFC-365mfc', Category.F_GAS),
    'HFC-4310mee': SpeciesID('HFC-4310mee', Category.F_GAS),
    'NF3': SpeciesID('NF3', Category.F_GAS),
    'SF6': SpeciesID('SF6', Category.F_GAS),
    'SO2F2': SpeciesID('SO2F2', Category.F_GAS),
    # aerosols, ozone, and their precursors
    'Sulfur': SpeciesID('Sulfur', Category.SULFUR),
    'BC': SpeciesID('BC', Category.BC),
    'OC': SpeciesID('OC', Category.OC),
    'NH3': SpeciesID('NH3', Category.OTHER_AEROSOL),
    'VOC': SpeciesID('VOC', Category.REACTIVE_GAS),
    'CO': SpeciesID('CO', Category.REACTIVE_GAS),
    'NOx': SpeciesID('NOx', Category.REACTIVE_GAS),
    'ari': SpeciesID('Aerosol-Radiation Interactions', Category.AEROSOL_RADIATION_INTERACTIONS),
    'aci': SpeciesID('Aerosol-Cloud Interactions', Category.AEROSOL_CLOUD_INTERACTIONS),
    'ozone': SpeciesID('Ozone', Category.OZONE),
    # Contrails and precursors
    'NOx_aviation': SpeciesID('NOx Aviation', Category.NOX_AVIATION),
    'contrails': SpeciesID('Contrails', Category.CONTRAILS),
    # other minor anthropogenic
    'LAPSI': SpeciesID('Light absorbing particles on snow and ice', Category.LAPSI),
    'H2O_stratospheric': SpeciesID('H2O Stratospheric', Category.H2O_STRATOSPHERIC),
    'land_use': SpeciesID('Land Use', Category.LAND_USE),
    # natural
    'solar': SpeciesID('Solar', Category.SOLAR),
    'volcanic': SpeciesID('Volcanic', Category.VOLCANIC)
}

In [ ]:
scenarios_to_include=['ssp245']

In [ ]:
df_emis = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
df_forc = pd.read_csv('../data/forcing/table_A3.3_historical_ERF_1750-2019_best_estimate.csv')

In [ ]:
emitted_species = [
    'CO2_FFI', 'CO2_AFOLU', 'CH4', 'N2O',
    'Sulfur', 'BC', 'OC', 'NH3', 'NOx', 'VOC', 'CO',
    'CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115',
    'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br',
    'Halon-1211', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18',
    'NF3', 'SF6', 'SO2F2',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32', 
    'HFC-365mfc', 'HFC-4310mee', 'NOx_aviation']
forced_species = ['solar', 'volcanic']
from_other_species = ['CO2', 'ari', 'aci', 'ozone', 'contrails', 'LAPSI', 'H2O_stratospheric', 'land_use']

species_to_include = emitted_species + forced_species + from_other_species

In [ ]:
scenarios = []
for iscen, scenario in enumerate(scenarios_to_include):
    list_of_species = []
    for ispec, species in enumerate(emitted_species):
        species_rcmip_name = species.replace("-", "")
        if species == 'NOx_aviation':
            species_rcmip_name = 'NOx|MAGICC Fossil and Industrial|Aircraft'
        elif species == 'CO2_FFI':
            species_rcmip_name = 'CO2|MAGICC Fossil and Industrial'
        elif species == 'CO2_AFOLU':
            species_rcmip_name = 'CO2|MAGICC AFOLU'
        emis_in = df_emis.loc[
            (df_emis['Scenario']==scenario) & (df_emis['Variable'].str.endswith("|"+species_rcmip_name)) & 
            (df_emis['Region']=='World'), '1750':'2030'
        ].interpolate(axis=1).values.squeeze()

        # CO2 and N2O units need to behave: TODO, sort this out
        if species in ('CO2_FFI', 'CO2_AFOLU', 'N2O'):
            emis_in = emis_in / 1000
        list_of_species.append(Species(species_ids[species], emissions=emis_in))
        #print(species_rcmip_name, emis_in.shape)
        
    # solar and volcanic forcing still a little hacky
    solar_forcing = np.zeros(281)
    solar_forcing[:270] = df_forc['solar'].values
    volcanic_forcing = np.zeros(281)
    volcanic_forcing[:270] = df_forc['volcanic'].values
    volcanic_forcing[269:] = np.linspace(1, 0, 12) * volcanic_forcing[269]
    list_of_species.append(Species(species_ids['solar'], forcing=solar_forcing))
    list_of_species.append(Species(species_ids['volcanic'], forcing=volcanic_forcing))
    
    # add derived species: at this stage just a declaration that we want them
    for species in from_other_species:
        list_of_species.append(Species(species_ids[species]))
        
    scenarios.append(Scenario(scenario, list_of_species))

In [ ]:
iterations = 2000
batch_size = 500

## Define parameters and starting values

In [ ]:
bounds = np.array(
    [
        [0.2, 4],  # kappa1
        [1.5, 7],  # kappa2
        [0.3, 3],  # kappa3
        [1.6, 6.0],  # c1
        [1.6, 15],   # c2
        [10, 290],   # c3
        [0.8, 2.0],  # epsilon
        [0.8, 3.0],   # gamma
        [0.5, 2.5],   # sigma eta
        [0.25, 1],   # sigma xi
        [0.75, 1.25]  # scale co2
    ]
)

jump = np.array(
    [0.05, 0.05, 0.05, 0.5, 0.1, 7, 0.03, 0.1, 0.1, 0.04, 0.01]
)

params = scipy.stats.uniform.rvs(loc=bounds[:,0], scale=bounds[:,1]-bounds[:,0], size=(batch_size, 11))

In [ ]:
species_index = {}
for i in range(len(species_to_include)):
    species_index[species_to_include[i]] = i

In [ ]:
0.5+0.5*np.tanh(8-50*np.arange(0.1, 0.21, 0.01))

In [ ]:
pl.plot(np.arange(0,0.5,0.01), 0.5+0.5*np.tanh(8-50*np.arange(0, 0.5, 0.01)))

In [ ]:
NINETY_TO_ONESIGMA=scipy.stats.norm.ppf(0.95)
ecs_dist = scipy.stats.skewnorm(8.82185594, loc=1.95059779, scale=1.55584604)
tcr_dist = scipy.stats.norm(loc=1.2, scale=0.6/NINETY_TO_ONESIGMA)
ohc_dist = scipy.stats.norm(loc=434.3730268913012, scale=73.70562981598447)
temp_9514_dist = scipy.stats.skewnorm(-1.65506091, loc=0.92708099, scale=0.12096636)
temp_rmse_dist = scipy.stats.halfnorm(scale=0.11)

def rmse(obs, mod):
    return np.sqrt(np.sum((obs-mod)**2)/len(obs))

In [ ]:
df_gmst = pd.read_csv('../data/forcing/AR6_GMST.csv')
gmst = df_gmst['gmst'].values

In [ ]:
run_config = RunConfig(ch4_lifetime_method=CH4LifetimeMethod.AERCHEMMIP)

# for all except temperature, the full time series is not important.
temp_rmse_out = np.ones((iterations, batch_size)) * np.nan
temp_9514_out = np.ones((iterations, batch_size)) * np.nan
ohc_out = np.ones((iterations, batch_size)) * np.nan
ecs_out = np.ones((iterations, batch_size)) * np.nan
tcr_out = np.ones((iterations, batch_size)) * np.nan

# nothing to start from
likelihood = np.zeros(500)

seedgen = 370160
for i in tqdm(range(iterations)):
    # calculate random step size
    step = scipy.stats.norm.rvs(loc=0, scale=jump*np.ones((batch_size, 11)))
    
    # pre-calculate replacement values if we go off a cliff (wasteful but likely more efficient than looping)
    redraw = scipy.stats.uniform.rvs(loc=bounds[:,0], scale=bounds[:,1]-bounds[:,0], size=(batch_size, 11))
    newparams = params + step
    
    # fill in places where we go off a cliff
    newparams[newparams[:,:] > bounds[:,1]] = redraw[newparams[:,:] > bounds[:,1]]
    newparams[newparams[:,:] < bounds[:,0]] = redraw[newparams[:,:] < bounds[:,0]]
    
    configs = []
    for iconf in range(batch_size):
        config_name = f"{iconf}"

        # Climate response configs
        climate_response = ClimateResponse(
            ocean_heat_capacity = [params[iconf, 3], params[iconf, 4], params[iconf, 5]],
            ocean_heat_transfer = [params[iconf, 0], params[iconf, 1], params[iconf, 2]],
            deep_ocean_efficacy = params[iconf, 6],
            gamma_autocorrelation = params[iconf, 7],
            sigma_eta = params[iconf, 8],
            sigma_xi = params[iconf, 9],
            stochastic_run = False,
            seed=seedgen+iconf*163
        )
        
        # Species configs
        # get defaults
        species_config = [copy.copy(species_config_from_default(species)) for species in species_to_include]
        
        # methane lifetime params (not varied, defaults are baked in)
        species_config[2].lifetime = 10.788405534387858
        species_config[2].natural_emissions_adjustment = 19.019783117809567
        species_config[2].soil_lifetime = 185
        
        species_config[54].scale = params[iconf, 10]
        
        # volcanic efficacy is something like 0.6
        species_config[53].efficacy = 0.6
        
        configs.append(Config(config_name, climate_response, species_config))
        
    fair = FAIR(scenarios, configs, run_config=run_config)
    fair.run()
    
    
    fair.calculate_ocean_heat_content_change()
    
    #temp_rmse_out = np.ones((iterations, batch_size)) * np.nan
    temp_9514_out[i, :] = fair.temperature[245:265, 0, :, 0, 0].mean(axis=0) - fair.temperature[100:151, 0, :, 0, 0].mean(axis=0)
    
    
    for iconf in range(batch_size):
        ebm = EnergyBalanceModel(
            ocean_heat_capacity = np.array([params[iconf, 3], params[iconf, 4], params[iconf, 5]]),
            ocean_heat_transfer = np.array([params[iconf, 0], params[iconf, 1], params[iconf, 2]]),
            deep_ocean_efficacy = params[iconf, 6],
            gamma_autocorrelation = params[iconf, 7],
            sigma_eta = params[iconf, 8],
            sigma_xi = params[iconf, 9],
            forcing_4co2 = 2* 3.934 * params[iconf, 10]
        )
        ebm.emergent_parameters()
        ecs_out[i, iconf] = ebm.ecs
        tcr_out[i, iconf] = ebm.tcr
        temp_rmse_out[i, iconf] = rmse(gmst[:171], fair.temperature[100:271, 0, iconf, 0, 0]-fair.temperature[100:151, 0, iconf, 0, 0].mean(axis=0))
        
    ohc_out[i, :] = 1e-21*(fair.ocean_heat_content_change[268, 0, :, 0, 0]-fair.ocean_heat_content_change[221, 0, :, 0, 0])
#    print(ohc_out[i, :])
#    print(ecs_out[i, :])
#    print(tcr_out[i, :])
#    print(temp_9514_out[i, :])
    likelihood_old = likelihood
    
    likelihood = (
        ohc_dist.pdf(ohc_out[i, :]) *
        temp_9514_dist.pdf(temp_9514_out[i, :]) * 
        ecs_dist.pdf(ecs_out[i, :]) *
        tcr_dist.pdf(tcr_out[i, :]) *
        (0.5+0.5*np.tanh(8-50*(temp_rmse_out[i, :])))
    )
    
    if i==0:
        likelihood_old = likelihood
    likelihood_old[likelihood_old==0] = 1e-100  # prevents divide by zero, ensures next step is almost always accepted
    accept_probability = likelihood / likelihood_old
#    accept_probability[np.isinf(accept_probability)] = 1  # force a move when minus infinity crops up, as it likely means
#                                                          # the likelihood in the previous iteration was zero, so anything
#                                                          # drawn at random will be no worse and possibly be an improvement
    #accept = likelihood > likelihood_old
    coinflip = scipy.stats.uniform.rvs(size=batch_size)
    
    params[accept_probability >= coinflip, :] = newparams[accept_probability >= coinflip, :]
    likelihood[accept_probability < coinflip] = likelihood_old[accept_probability < coinflip]
    #likelihood < like
    
    #params[likelihood > likelihood_old, :] = newparams[likelihood > likelihood_old, :]
     
    
#     co2_out[batch_start:batch_end] = fair.concentration_array[264, 0, :, 54, 0]
#     fari_out[batch_start:batch_end] = fair.forcing_array[255:265, 0, :, 55, 0].mean(axis=0)
#     faci_out[batch_start:batch_end] = fair.forcing_array[255:265, 0, :, 56, 0].mean(axis=0)
#     fo3_out[batch_start:batch_end] = fair.forcing_array[269, 0, :, 57, 0]
    
#     # at this point dump out some batch output and put the constraining in another sheet
#     temp_out[:, batch_start:batch_end] = fair.temperature[100:, 0, :, 0, 0]

In [ ]:
pl.plot(temp_9514_out);

In [ ]:
temp_9514_out[:,70]

In [ ]:
ecs_out[:,70]

In [ ]:
tcr_out[:,70]

In [ ]:
ohc_out[:,70]

In [ ]:
likelihood

In [ ]:
ohc_dist.pdf(ohc_out[i, :])

In [ ]:
pl.hist(ecs_out[-1,:], bins=np.arange(0,20,0.2))

In [ ]:
temp_rmse_out[:,70]

In [ ]:
np.percentile(ecs_out[-1,:], (5, 50, 95))

In [ ]:
accept_probability

In [ ]:
np.sum(accept_probability >= coinflip)/500

In [ ]:
coinflip

In [ ]:
0.5+0.5*np.tanh(8-50*(temp_rmse_out[i, :]))

In [ ]:
likelihood

In [ ]:
pl.hist(temp_9514_out[-1,:], bins=np.arange(0,2.6,0.1))

In [ ]:
pl.hist(temp_rmse_out[-1,:], bins=np.arange(0,0.7,0.02))